# Simple 2 bus Example

This tutorial shows how to perform three phase load flow in an unbalanced system.
<img src="pics/Two bus system.png" width="80%">

THree phase or unbalanced load flow is performed using the function 

runpp_3ph(net) 

This function needs to be explicitly imported, when unbalanced load flow calculation is desired

In [33]:
import pandapower as pp
from pandapower.pf.runpp_3ph import runpp_3ph

# Create Buses
After this import we can start by creating an empty network and two 110 kV buses 

# Create External Grid
For negative and zero sequence networks. 
The following data is needed in addition to a voltage reference.
- s_sc_max_mva
- rx_max
- r0x0_max
- x0x_max

Some of them needs to be added directly to PandapowerNet

In [34]:
net = pp.create_empty_network(sn_mva = 100 )
bus_eg  =  pp.create_bus(net, vn_kv = 110, name = "Ext. Grid")
bus_load  =  pp.create_bus(net, vn_kv = 110, name = "Load")

pp.create_ext_grid(net, bus=bus_eg, vm_pu= 1.0, name="Grid Connection",s_sc_max_mva=5000, rx_max=0.1)

# Needs to be added explicitly after create function
net.ext_grid["r0x0_max"] = 0.1
net.ext_grid["x0x_max"] = 1.0


You can see below, external grid has all positive and zero sequence parameters

In [35]:
net.ext_grid

,name,bus,vm_pu,va_degree,in_service,s_sc_max_mva,rx_max,r0x0_max,x0x_max
0,Grid Connection,0,1.0,0.0,True,5000.0,0.1,0.1,1.0


# Create a Standard Type
A line type should have the following parameters for unbalanced load flow:

- r_ohm_per_km
- x_ohm_per_km
- c_nf_per_km
- r0_ohm_per_km
- x0_ohm_per_km
- c0_nf_per_km
- max_i_ka

# Create a Line
After creating a line type, we can create several lines like the one below:

In [36]:
#Line Type created
pp.create_std_type(net, {"r0_ohm_per_km": 0.0848, "x0_ohm_per_km": 0.4649556, "c0_nf_per_km":\
    230.6,"max_i_ka": 0.963, "r_ohm_per_km": 0.0212, "x_ohm_per_km": 0.1162389,
             "c_nf_per_km":  230}, "example_type")

# Actual line created with the length of the line in km
pp.create_line(net, from_bus = bus_eg, to_bus = bus_load, length_km = 50.0, std_type="example_type")


0

# Create Unbalanced Load
We can create three phase loads by specifying P, Q values in each phase.
The default values are set to zero.

Connection type of load can be specified while creating a three phase load

**'wye' (Default)** - 3 phase phase-earth load

**'delta'** - 3 phase delta load

PS: When a balanced load is created using pp.create_load , the load is divided equally among the three phases.


In [37]:
#This function creates an unbalanced load
pp.create_asymmetric_load(net, bus_load, p_A_mw=50, q_A_mvar=50, p_B_mw=10, q_B_mvar=15,
                   p_C_mw=10, q_C_mvar=5)

# pp.create_load creates a balanced three phase load 


0

# add_zero_impedance_parameters
This function adds the zero sequence parameters in the standard type
of the element (trafo/line)

# runpp_3ph
This function performs the calculation and returns results in 
# result tables:
-res_element_3ph 

In [38]:
pp.add_zero_impedance_parameters(net)
count, V012_it, I012_it, ppci0, Y0_pu, Y1_pu, Y2_pu = runpp_3ph(net)
net.res_bus_3ph

,vmA_pu,vaA_degree,vmB_pu,vaB_degree,vmC_pu,vaC_degree,p_A_mw,q_A_mvar,p_B_mw,q_B_mvar,p_C_mw,q_C_mvar
0,0.967429,-1.051253,1.013028,-119.470654,1.019784,120.471400,68.55948,83.019541,22.840554,-23.79116,-17.677152,-12.140862
1,0.749574,-9.010213,1.091380,-124.105793,1.051243,125.894504,50.00000,50.000000,10.000000,15.00000,10.000000,5.000000


The convergence of load flow can be checked with the following:

In [39]:
net['converged']

True

# 3 Bus system with a transformer in between External Grid and Distribution system

<img src="pics/Three bus system.png" width="80%">

In [40]:
net = pp.create_empty_network(sn_mva=100)
bus_eg = pp.create_bus(net, 10)
bus_lv = pp.create_bus(net, 0.4)
bus_load = pp.create_bus(net, 0.4 )

pp.create_ext_grid(net, bus_eg, s_sc_max_mva=10000, rx_max=0.1)
net.ext_grid["r0x0_max"] = 0.1
net.ext_grid["x0x_max"] = 1.0

# Create Transformer.

Create a standard type and use it during transformer creation in the network

In [41]:
pp.create_std_type(net, {"sn_mva": 1.6,
            "vn_hv_kv": 10,
            "vn_lv_kv": 0.4,
            "vk_percent": 5.94892,
            "vkr_percent": 0.78125,
            "pfe_kw": 2.7,
            "i0_percent": 0.16875,
            "shift_degree": 0,
            "vector_group": 'Dyn',
            "tap_side": "lv",
            "tap_neutral": 0,
            "tap_min": -2,
            "tap_max": 2,
            "tap_step_degree": 0,
            "tap_step_percent": 2.5,
            "tap_phase_shifter": False,
            "vk0_percent": 5.94892, 
            "vkr0_percent": 0.78125, 
            "mag0_percent": 100,
            "mag0_rx": 0.,
            "si0_hv_partial": 0.9,}, name='1.6 MVA 10/0.4kV Distribution Transformer', element="trafo")
    
pp.create_transformer(net, bus_eg, bus_lv, std_type='1.6 MVA 10/0.4kV Distribution Transformer', parallel=1,tap_pos = 0 )

0

# Create line and loads 

Just like before create three phase lines and loads with zero sequence parameters

In [42]:
pp.create_std_type(net, {"r_ohm_per_km": 0.1941, "x_ohm_per_km": 0.07476991,
                    "c_nf_per_km": 1160., "max_i_ka": 0.421,
                    "endtemp_degree": 70.0, "r0_ohm_per_km": 0.7766,
                    "x0_ohm_per_km": 0.2990796,
                    "c0_nf_per_km":  496.2}, name="unsymmetric_line_type",element = "line")


pp.create_line(net, bus_lv, bus_load, length_km=0.5, std_type="unsymmetric_line_type")

#Creates a 3 phase wye load
pp.create_asymmetric_load(net, bus_load, p_A_mw=0.0300, q_A_mvar=0.0048, p_B_mw=0.0280, q_B_mvar=0.0036,
                               p_C_mw=0.027, q_C_mvar=0.0043,type='wye')

0

# Run Asymmetric load flow for the network

Same as before, add zero sequence parameters and run 3 phase load flow.


In [43]:
pp.add_zero_impedance_parameters(net)
count, V012_it, I012_it, ppci0, Y0_pu, Y1_pu, Y2_pu = runpp_3ph(net)
net.res_bus_3ph

,vmA_pu,vaA_degree,vmB_pu,vaB_degree,vmC_pu,vaC_degree,p_A_mw,q_A_mvar,p_B_mw,q_B_mvar,p_C_mw,q_C_mvar
0,1.000000,-0.000019,1.000000,-119.999989,1.000000,120.000007,0.033759,0.005014,0.032224,0.005856,0.032262,0.004106
1,0.998888,-0.208098,0.999052,-120.191927,0.999042,119.815207,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.928695,-0.907565,0.945274,-121.397996,0.948889,119.495072,0.030000,0.004800,0.028000,0.003600,0.027000,0.004300


# Sequence Frame results 
Sequence frame results are returned by runpp_3ph instead of using net result tables.

# Format of the array
# | V0: bus_eg ,   bus_hv ,   bus_load |
# | V1:  ...| 
# | V2:  ...|

In [44]:
# See the zero sequence voltage values
V012_it[0,:]

array([-0.00000000e+00+0.j        , -8.91079944e-05-0.00011733j,
       -1.03580736e-02+0.00144819j])